In [55]:
import numpy as np
import math
import pandas as pd

## Ввод необходимых констан, вектора узловых перемещений(Г.У.)

In [56]:
a = 1.4   
E = 1.1e11
D = 40e-3 ##Внешний диаметр
a1 = 30e-3 #ширина полки 
h1 = 10e-3 # высота полки
h2 = 8e-3 #  толщина полки
b = 40e-3 # ширина полки 

R = np.array([[1,2],[2,3],[3,4],[4,5],[5,6],[6,7],[6,8]])
n_el = 7
n_node = 8
X = np.array([0,0,1,
              1,1,1,
              1,1,1,
              1,1,1,
              1,1,1,
              1,1,1,
              1,0,1,
              1,0,1])
F = 900
M = 800
q1 = 800
q2 = 100

## Расчет длин, площадей и моментов инерции сечений, вектора узловых нагрузок

In [57]:
l1, l2, l3, l4, l5, l6, l7 = a/2, a/2, a/4, a/4, math.sqrt((a/2)**2 + (a/2)**2)/2, math.sqrt((a/2)**2 + (a/2)**2)/2, math.sqrt((a/4)**2 + (a/4)**2)
L = np.array([l1, l2, l3, l4, l5, l6, l7])
Angles = np.array([math.pi, -math.pi/2, 0, 0, -math.pi/4, -math.pi/4, -3*math.pi/4])

S_krug = math.pi*(D**2)/4
S_kvadr = a1*h1 + (b-h1)*h2

J_krug = math.pi*(D**4)/64
J_kvadr = 1.3912e-8

Q = np.array([0,0,0,
              q1*L[1]/2,0,q1*L[1]**2/12,
              q1*L[1]/2,0,-q1*L[1]**2/12,
              0,-F,0,
              -q2*L[4]/2 * math.cos(-math.pi/4), q2*L[4]/2 * math.sin(-math.pi/4), -q2*L[4]**2/12 - M,
              -q2 * (L[4]+L[5]) * math.cos(-math.pi/4)/2, q2 * (L[4]+L[5]) * math.sin(-math.pi/4)/2, 0,
              0, 0, q2*L[5]**2/12,
              0, 0, 0])

## Локальные матрицы жесткости и направляющих косинусов

In [58]:
def Create_Rotation_Matrix(angle):
    '''
    Функция собирает матрицу направляющих косинусов
    '''
    T = np.matrix([[math.cos(angle), math.sin(angle), 0, 0, 0, 0],
                   [-math.sin(angle), math.cos(angle), 0, 0, 0, 0],
                   [0, 0, 1, 0, 0, 0],
                   [0, 0, 0, math.cos(angle), math.sin(angle), 0],
                   [0, 0, 0, -math.sin(angle), math.cos(angle), 0],
                   [0, 0, 0, 0, 0, 1]])
    return T

Ниже вывожу матрицу напр. косинусов, чтобы вставить результаты в отчет

In [59]:
T3 = Create_Rotation_Matrix(Angles[6])
T3

matrix([[-0.70710678, -0.70710678,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.70710678, -0.70710678,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        , -0.70710678, -0.70710678,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.70710678, -0.70710678,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          1.        ]])

In [60]:
def Create_Local_Matrix(A,L,J,angle):
    '''
    Функция собирает локальную матрицы жесткости, а после считает глобальную матрицу жесткости
    каждого элемента, учитывая матрицу направляющих косинусов
    '''
    K_loc = np.array([[E*A/L, 0, 0, -E*A/L, 0, 0],
                      [0, 12*E*J/L**3, 6*E*J/L**2, 0, -12*E*J/L**3, 6*E*J/L**2],
                      [0, 6*E*J/L**2, 4*E*J/L, 0, -6*E*J/L**2, 2*E*J/L],
                      [-E*A/L, 0, 0, E*A/L, 0, 0],
                      [0, -12*E*J/L**3, -6*E*J/L**2, 0, 12*E*J/L**3, -6*E*J/L**2],
                      [0, 6*E*J/L**2, 2*E*J/L, 0, -6*E*J/L**2, 4*E*J/L]])
    #для вывода локальной матрицы.
    # print(K_loc*10**-6)
    T = Create_Rotation_Matrix(angle)
    K_loc = np.dot(np.dot(T.T, K_loc),T)
    return K_loc

In [61]:
def Create_H_Matrix(R,n_el,n_node):
    '''
    Функция собирает матрицу соответствия
    '''
    k = -1
    H = np.zeros((6*n_el, n_node*3))
    for i in range(n_el):
        for j in range(2):
            a = R[i][j]
            H[k+1][3*a-3] = 1
            H[k+2][3*a-2] = 1
            H[k+3][3*a-1] = 1
            k+=3
    
    return H

Ниже вывожу матрицу соответствия, чтобы вставить результаты в отчет. Можно сохранить в xlsx, чтобы удобнее вставлять в отчет.

In [62]:
H = Create_H_Matrix(R,n_el,n_node)
df = pd.DataFrame(H)
# df.to_excel("H.xlsx")
df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Аналогично вывожу Глобальную матрицу жесткости каждого элемента. Внутри Create_Local_Matrix можно расскоментить, чтобы одновременно выводить и локальную матрицу жесткости элеменнтов 

In [63]:
k_loc = Create_Local_Matrix(S_krug, L[5], J_krug, Angles[5])
print("_____________________________________")
print(k_loc*10**-6)

_____________________________________
[[ 1.40317383e+02 -1.38949545e+02  2.39371652e-01 -1.40317383e+02
   1.38949545e+02  2.39371652e-01]
 [-1.38949545e+02  1.40317383e+02  2.39371652e-01  1.38949545e+02
  -1.40317383e+02  2.39371652e-01]
 [ 2.39371652e-01  2.39371652e-01  1.11706771e-01 -2.39371652e-01
  -2.39371652e-01  5.58533855e-02]
 [-1.40317383e+02  1.38949545e+02 -2.39371652e-01  1.40317383e+02
  -1.38949545e+02 -2.39371652e-01]
 [ 1.38949545e+02 -1.40317383e+02 -2.39371652e-01 -1.38949545e+02
   1.40317383e+02 -2.39371652e-01]
 [ 2.39371652e-01  2.39371652e-01  5.58533855e-02 -2.39371652e-01
  -2.39371652e-01  1.11706771e-01]]


## Глобальная матрица жесткости конструкции

In [64]:
def Create_Global_Matrix(L, Angles, n_el):
    '''
    Функция собирает квазидиагональную матрицу жесткости конструкции.
    '''
    K = np.zeros((n_el*6, n_el*6))
    for i in range(n_el):
        if i == 0:
            k_loc = Create_Local_Matrix(S_kvadr,L[i],J_kvadr,Angles[i])
            K[i*6:6+i*6,i*6:6+i*6] = k_loc
            continue
        if i == n_el-1:
            k_loc = Create_Local_Matrix(S_kvadr,L[i],J_kvadr,Angles[i])
            K[i*6:6+i*6,i*6:6+i*6] = k_loc
            continue
        k_loc = Create_Local_Matrix(S_krug,L[i],J_krug,Angles[i])
        K[i*6:6+i*6,i*6:6+i*6] = k_loc
    return K

In [65]:
K = Create_Global_Matrix(L, Angles, n_el);

Ниже вывожу и сохраняю квазидиагональную матрицу жесткости конструкции, чтобы вставить в отчет

In [66]:
df = pd.DataFrame(K*10**-6)
# df.to_excel("K.xlsx")

А сейчас рассчитывается глобальная матрица жесткости всей конструкции.

In [67]:
K_glob = np.dot(np.dot(H.T, K),H)

Ниже вывожу и сохраняю глобальную матрицу жесткости конструкции, чтобы вставить в отчет

In [68]:
df = pd.DataFrame(K_glob*10**-6)
# df.to_excel("K_glob.xlsx")

## Расчет перемещений и нагрузок

Далее учту граничные условия на степени свободы для глобальной матрицы жесткости

In [69]:
K = K_glob.copy()
for i in range(X.shape[0]):
    if X[i] == 0:
        K[i,::] = 0
        K[::,i] = 0
        K[i,i] = 1

И сохраню получившиеся результаты, чтобы вставить в отчет

In [70]:
df = pd.DataFrame(K*10**-6)
# df.to_excel("K_after_BC.xlsx")

Далее будет рассчет Вектора перемещений и нагрузок

In [71]:
Inv_K = np.linalg.inv(K) 

In [72]:
U = np.dot(Inv_K, Q)
df = pd.DataFrame(U)
df*10**3

,0
0,0.000000
1,0.000000
2,15.840811
3,0.005981
4,-11.715919
5,18.529458
6,11.665121
7,-11.715834
8,13.438296
9,11.664988


In [73]:
F = np.dot(K_glob,U)
df = pd.DataFrame(F)
df

,0
0,-5.075000e+02
1,-1.679384e+01
2,1.136470e-09
3,2.800000e+02
4,0.000000e+00
5,3.266667e+01
6,2.800000e+02
7,9.313226e-10
8,-3.266667e+01
9,0.000000e+00
